In [1]:
import pandas as pd
train=pd.read_csv('./train.csv') #Train Data  

In [2]:
# jeju=train[train['CARD_SIDO_NM'].isin(['제주'])]
# jeju

In [3]:
!pip install lightgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime
import sys
import os
import gc #Garbage Collector
import numpy as np # linear algebra
import time
import warnings

In [4]:
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

pandas: 1.0.1
numpy: 1.18.1
Python: 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]


In [5]:
train.keys()

Index(['Unnamed: 0', 'REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM',
       'HOM_SIDO_NM', 'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT',
       'AMT', 'CNT'],
      dtype='object')

jeju_card_usage

-REG_YYMM 년월	

-CARD_SIDO_NM 
카드이용지역_시도 (가맹점 주소 기준)	

-CARD_CCG_NM 카드이용지역_시군구 (가맹점 주소 기준)

-STD_CLSS_NM	업종명

-HOM_SIDO_NM	거주지역_시도 (고객 집주소 기준)

-HOM_CCG_NM 거주지역_시군구 (고객 집주소 기준)

-AGE	연령대

-SEX_CTGO_CD	성별 (1: 남성, 2: 여성)

-FLC	가구생애주기 
(1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)

-CSTMR_CNT 이용고객수 (명)

-AMT	이용고객수 (명)

-CNT 이용건수 (건)


## 결측치

In [6]:
train=train.drop(['HOM_CCG_NM','CARD_CCG_NM','Unnamed: 0'],axis=1)

In [7]:
train.isnull().any()

REG_YYMM        False
CARD_SIDO_NM    False
STD_CLSS_NM     False
HOM_SIDO_NM     False
AGE             False
SEX_CTGO_CD     False
FLC             False
CSTMR_CNT       False
AMT             False
CNT             False
dtype: bool

In [8]:
train

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,서울,한식 음식점업,전북,50s,1,4,5,303000,5
1,201901,서울,비알콜 음료점업,경기,50s,2,4,31,464900,45
2,201901,경북,한식 음식점업,경북,30s,1,1,38,1158600,51
3,201901,경남,체인화 편의점,제주,20s,1,1,15,906900,125
4,201901,경남,기타음식료품위주종합소매업,경남,20s,1,1,7,407040,25
...,...,...,...,...,...,...,...,...,...,...
99858,202003,충남,체인화 편의점,광주,20s,1,1,3,22400,7
99859,202003,경기,차량용 주유소 운영업,인천,40s,1,3,50,8023988,97
99860,202003,서울,슈퍼마켓,서울,20s,1,1,24,429210,43
99861,202003,부산,체인화 편의점,서울,40s,1,3,13,565540,61


In [9]:
#날짜 전처리
def grab_year(data):
    data = str(data)
    return int(data[:4])

def grab_month(data):
    data = str(data)
    return int(data[4:])

In [10]:
train['year'] = train['REG_YYMM'].apply(lambda x: grab_year(x))
train['month'] = train['REG_YYMM'].apply(lambda x: grab_month(x))

In [11]:
train = train.drop(['REG_YYMM'], axis=1)

In [12]:
train.shape, type(train)

((99863, 11), pandas.core.frame.DataFrame)

In [13]:
train.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,서울,한식 음식점업,전북,50s,1,4,5,303000,5,2019,1
1,서울,비알콜 음료점업,경기,50s,2,4,31,464900,45,2019,1
2,경북,한식 음식점업,경북,30s,1,1,38,1158600,51,2019,1
3,경남,체인화 편의점,제주,20s,1,1,15,906900,125,2019,1
4,경남,기타음식료품위주종합소매업,경남,20s,1,1,7,407040,25,2019,1


In [15]:
train.to_csv("train_edit.csv",header=True,index=True) #csv